In [1]:
import tensorflow as tf
import numpy as np
import numpy.random as rng
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\prash\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [37]:
#Creating the dataset 
inshape=784 #input size
hiddenshape=20 # No of hidden units 
outshape=10 # Output size 

batch_size = 50

#Training parameters 
iterations=3000
i=tf.Variable(0,name='loop_i')
eta=tf.constant(0.00001,dtype=tf.float64)

#Training data inputs
'''x=tf.placeholder(tf.float64,[None,inshape], name = 'x')
y=tf.placeholder(tf.float64,[None,outshape], name = 'y')
'''
x=tf.placeholder(tf.float64,[batch_size,784])
y=tf.placeholder(tf.float64,[batch_size,10])

#Linear network
#B=tf.random_normal([hiddenshape,outshape],mean=0.0,stddev=1.0,dtype=tf.float64)
#w_1=tf.random_normal([inshape,hiddenshape],mean=0.0,stddev=1.0,dtype=tf.float64)
#w_2=tf.random_normal([hiddenshape,outshape],mean=0.0,stddev=1.0,dtype=tf.float64)

#parameters for feedback alignment
B = tf.Variable(rng.randn(hiddenshape, outshape))
w_1 = tf.Variable(rng.randn(hiddenshape, inshape))
w_2= tf.Variable(rng.randn(outshape, hiddenshape))

hidout=tf.matmul(w_1,x)
y_pred_fa=tf.matmul(w_2,hidout) 

e_fa=y_pred_fa-y
cost_fa = tf.reduce_sum(tf.pow(e_fa, 2))/2
Be=tf.matmul(B,e_fa)

dLw1=tf.matmul(Be,tf.transpose(x))
dLw2=tf.matmul(e_fa,tf.transpose(hidout))

#Parameters for back propogation

w1bp=tf.Variable(hiddenshape,outshape)
w2bp=tf.Variable(outshape,hiddenshape)

w1bp=w_1 
w2bp=w_2

hidout_bp=tf.matmul(w1bp,x)
y_pred_bp=tf.matmul(w2bp,hidout_bp)

e_bp=y_pred_bp-y
cost_bp = tf.reduce_sum(tf.pow(e_bp, 2))/2
We=tf.matmul(tf.transpose(w2bp),e_bp)

dLw1bp=tf.matmul(We,tf.transpose(x))
dLw2bp=tf.matmul(e_bp,tf.transpose(hidout_bp))

#Training data generation
#x=tf.random_normal([inshape,1],mean=0.0,stddev=1.0,dtype=tf.float64)
#y=tf.matmul(T,x)
#T=tf.random_normal([outshape,inshape],mean=0.0,stddev=1.0,dtype=tf.float64)
T = rng.randn(outshape, inshape)
#def traindata(T, batch_size):
 #   train_x = rng.randn(T.shape[1], batch_size)
  #  train_y = np.dot(T, train_x)
   # return (train_x, train_y)''''

ValueError: Dimensions must be equal, but are 784 and 50 for 'MatMul_25' (op: 'MatMul') with input shapes: [20,784], [50,784].

In [8]:
def updatefa(w_1,w_2):    
    #gradients for a loss L 
    #Update
    #w_1=tf.subtract(w_1,tf.multiply(eta,tf.transpose(dLw1)))
    #w_2=tf.subtract(w_2,tf.multiply(eta,tf.transpose(dLw2))) 
    new_w1 = w_1.assign(w_1-tf.multiply(eta,dLw1))
    new_w2 = w_2.assign(w_2-tf.multiply(eta,dLw2)) 
    return new_w1, new_w2 

def updatebp(w1bp,w2bp):
    new_w1bp=w1bp.assign(w1bp-tf.multiply(eta,dLw1bp))
    new_w2bp=w2bp.assign(w2bp-tf.multiply(eta,dLw2bp))
    return new_w1bp,new_w2bp


In [9]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for idx in range(iterations):
        (train_x, train_y) = mnist.train.next_batch(batch_size)
        #sess.run(update(w_1, w_2), feed_dict = {x:train_x, y:train_y})
        #for jj in range(batch_size):
        sess.run(updatefa(w_1,w_2), feed_dict={x: train_x, y: train_y})
        sess.run(updatebp(w1bp,w2bp),feed_dict={x: train_x, y: train_y})
        cfa = sess.run(cost_fa, feed_dict={x: train_x, y: train_y})
        cbp = sess.run(cost_bp, feed_dict={x: train_x, y: train_y})
        if idx % 50==0:
            cfa = sess.run(cost_fa, feed_dict={x: train_x, y: train_y})
            cbp = sess.run(cost_bp, feed_dict={x: train_x, y: train_y})
            print("iteration: %d Feedbackalignment: %f\tBackprop:%f"%(idx,cfa,cbp))
            

ValueError: Cannot feed value of shape (30, 784) for Tensor 'x_1:0', which has shape '(784, ?)'

In [18]:
x=tf.placeholder(tf.float64,[None,inshape], name = 'x')
y=tf.placeholder(tf.float64,[None,outshape])
sess=tf.Session()
sess.run(updatefa(w_1, w_2), feed_dict = {x:train_x, y:train_y})

InvalidArgumentError: You must feed a value for placeholder tensor 'x_1' with dtype double and shape [784,?]
	 [[Node: x_1 = Placeholder[dtype=DT_DOUBLE, shape=[784,?], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'x_1', defined at:
  File "C:\Users\prash\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\prash\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\prash\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\prash\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\prash\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\prash\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C:\Use

In [34]:
#x=tf.placeholder(tf.float64,[None,784])
#y=tf.placeholder(tf.float64,[None,10])

x=tf.placeholder(tf.float64,[batch_size,784])
y=tf.placeholder(tf.float64,[batch_size,10])
x:train_x
y:train_y
np.shape(train_y)


(30, 10)

In [35]:
batch_size

50